##Install requirements packages

In [1]:
# !pip install transformers
# !pip install fastBPE
# !pip install fairseq
# # Install the vncorenlp python wrapper
# !pip install vncorenlp
# # Download VnCoreNLP-1.1.1.jar & its word segmentation component (i.e. RDRSegmenter) 
# !mkdir -p vncorenlp/models/wordsegmenter
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
# !wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
# !mv VnCoreNLP-1.1.1.jar vncorenlp/ 
# !mv vi-vocab vncorenlp/models/wordsegmenter/
# !mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

In [2]:
!ls

'Deep learning.ipynb'	   Train_with_embedding.ipynb  'model cbow'
'Machine learning.ipynb'   correct_teencode_stopword
 Train.ipynb		   dataset


**Install VnCoreNLP**

In [4]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("./vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

**Install pre-trained PhoBERT**

In [5]:
# !wget https://public.vinai.io/PhoBERT_base_transformers.tar.gz
# !tar -xzvf PhoBERT_base_transformers.tar.gz

In [6]:
# !wget https://public.vinai.io/PhoBERT_base_fairseq.tar.gz
# !tar -xzvf PhoBERT_base_fairseq.tar.gz

**Load Phobert**

In [32]:
from fairseq.data.encoders.fastbpe import fastBPE
from fairseq.data import Dictionary
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--bpe-codes', 
    default="./PhoBERT_base_transformers/bpe.codes",
    required=False,
    type=str,
    help='path to fastBPE BPE'
)
args, unknown = parser.parse_known_args()
bpe = fastBPE(args)

# Load the dictionary
vocab = Dictionary()
vocab.add_from_file("./PhoBERT_base_transformers/dict.txt")

Loading codes from ./PhoBERT_base_transformers/bpe.codes ...
Read 64000 codes from the codes file.


# Data preparing

In [8]:
# (train_text)

In [9]:
import tensorflow as tf
import pandas as pd 
import numpy as np
from string import digits
from collections import Counter
from pyvi import ViTokenizer
from gensim.models.word2vec import Word2Vec
from keras.utils.np_utils import to_categorical
import os 

%matplotlib inline

2022-05-01 09:51:45.186861: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-01 09:51:45.186894: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [10]:
PATH = ""
data_train = pd.read_csv(PATH+"dataset/vlsp_sentiment_train.csv", sep='\t')
data_train.columns =['Class', 'Data']
data_test = pd.read_csv(PATH+"dataset/vlsp_sentiment_test.csv", sep='\t')
data_test.columns =['Class', 'Data']

In [11]:
import bogo
import regex as re
import itertools
def map_to_unicode():
    dic = {}
    char1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'.split(
        '|')
    charutf8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ".split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic

def covert_unicode(txt):
    dicchar = map_to_unicode()
    return re.sub(
        r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ',
        lambda x: dicchar[x.group()], txt)
  
def preprocess(document):
    if type(document) is not str:
      document = str(document)
    # convert to lower case
    document = document.lower()

    #remove
  
    # remove html character
    document = re.sub(r'<[^>]*>', '', document)

    #convert all char to unicode
    document = covert_unicode(document)

    #remove adjacent identical characters
    document = ''.join(c[0] for c in itertools.groupby(document))

    #uwf=>ừ,....
    document = bogo.process_sequence(document)

    # remove error character
    document = re.sub(r'[^\s\wáàảãạăắằẳẵặâấầẩẫậéèẻẽẹêếềểễệóòỏõọôốồổỗộơớờởỡợíìỉĩịúùủũụưứừửữựýỳỷỹỵđ_]',' ',document)

    # remove multiple space character
    document = re.sub(r'\s+', ' ', document).strip()

    return document

In [12]:
data_train['Data'] = data_train['Data'].map(lambda x: preprocess(x))
data_train['Data']

0       mình đã dùng anywhere thế hệ đầu quả là đầy th...
1       quan tâm nhất là độ trễ có cao không dùng thi ...
2       dag xài con cùi bắp 98k pin trâu mỗi tội đánh ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       đang xài con m175 cùi mía nhà xài nhiều chuột ...
                              ...                        
5095    mình mua máy về đc 1 ngày mà điện thoại khác g...
5096    có bạn nào dùng f1ư ko mình dùng m cảm thấy qu...
5097    dùng ôp mà bộ nhớ 4gb thì k chơi games đc đâu ...
5098    sao tui thích xài hàng ôp mà lựa toàn mấy đứa ...
5099    mới mở hộp oy mở vào camera mà đã có ảnh chụp ...
Name: Data, Length: 5100, dtype: object

In [13]:
data_test['Data'] = data_test['Data'].map(lambda x: preprocess(x))
data_test['Data']

0       nói thiệt là mình thì thì chuột nào mình cũng ...
1       đang dùng mx1 cũng ngon nhưng chưa đầy năm mà ...
2       chưa thấy đc điểm thuyết phục để mua nhất là v...
3       những phần xem báo tra cứu bản đồ dịch vụ dùng...
4       đúng là mua ở việt nam không ứng dụng được gì ...
                              ...                        
1045                                              30 củ à
1046    aple bán dc thi samsung cũng lời nhiêu người k...
1047    có thể giúp android vượt trội so với ios chớ c...
1048    mẹ mình từng sang đài loan và có mua 1 cái iph...
1049    tùng minh nguyễn điện thoại của vk bị như này ...
Name: Data, Length: 1050, dtype: object

In [14]:
import json
import re
import itertools
import unidecode
import bogo

def read_file(file_path):
    fi = open(file_path, 'r', encoding='utf-8')
    ls = fi.readlines()
    return ls

'''load dictionary to mapping word'''
vowel_file = open(PATH+'correct_teencode_stopword/vietnamese_vowel.json', encoding='utf-8')
vowel_dic = json.load(vowel_file)

short_word_file = open(PATH+'correct_teencode_stopword/short_word.json', encoding='utf-8')
short_word_dic = json.load(short_word_file)

single_word_dic = read_file(PATH+'correct_teencode_stopword/unidecode_vietnamese_dic.txt')
single_word_dic = [re.sub('\n','', s) for s in single_word_dic]


def replace_one_one(word, dictionary):
    new_word = dictionary.get(word,word)
    return new_word
def correct_vowel(sent, vowel_dictionary):
    '''mapping a`, a\ --> à, ....'''
    words = sent.split()
    pattern = r'[aăâeêuưiyoôơ][.`~?\']'
    sent = ""
    for word in words:
        p = re.search(pattern, word)
        new_word = word
        if p:
            idx = p.span()
            replace_vowel = vowel_dictionary[word[idx[0]]][word[idx[0] + 1]]
            new_word = re.sub(pattern, replace_vowel, new_word)
        sent += new_word + ' '
    return sent

def correct_teencode(sent):
    sent = preprocess(sent)
    sent = correct_vowel(sent, vowel_dic)

    words = sent.split()
    sent = ""
    for word in words:
        new_word = ""
        if word[-1] in [',',';']:
            new_word = replace_one_one(word[:-1], short_word_dic)
            sent += new_word + word[-1]
        else:
            new_word = replace_one_one(word, short_word_dic)
            sent += new_word
        sent += ' '
    return sent[:-1]

In [15]:
import copy
data_fixteencode = copy.deepcopy(data_train)
data_fixteencode['Data'] = data_fixteencode['Data'].map(lambda x: correct_teencode(x))

In [16]:
data_fixteencode['Data']

0       mình đã dùng anywhere thế hệ đầu quả là đầy th...
1       quan tâm nhất là độ trễ có cao không dùng thi ...
2       dag xài con cùi bắp 98k pin trâu mỗi tội đánh ...
3       logitech chắc hàng phải tiền triệu trở lên dùn...
4       đang xài con m175 cùi mía nhà xài nhiều chuột ...
                              ...                        
5095    mình mua máy về được 1 ngày mà điện thoại khác...
5096    có bạn nào dùng f1ư không mình dùng m cảm thấy...
5097    dùng ôp mà bộ nhớ 4gb thì không chơi games đượ...
5098    sao tui thích xài hàng ôp mà lựa toàn mấy đứa ...
5099    mới mở hộp rồi mở vào camera mà đã có ảnh chụp...
Name: Data, Length: 5100, dtype: object

In [17]:
data_train = pd.concat([data_fixteencode, data_train])
data_train.shape

(10200, 2)

In [18]:
data_train = data_train.drop_duplicates()
data_train = data_train.dropna()
data_train['Class'].value_counts()

 0    2277
-1    2125
 1    2007
Name: Class, dtype: int64

In [19]:
y_train = data_train.iloc[:, 0].values
X_train = data_train.iloc[:, 1].values
y_train = y_train + 1

In [20]:
new_reviews = []
for idx,data in enumerate(X_train):
    text = rdrsegmenter.tokenize(data)
    text = ' '.join([' '.join(x) for x in text])
    new_reviews.append(text)
X_train = new_reviews

In [21]:
print(len(y_train))
len(X_train)

6409


6409

In [22]:
y_val = data_test.iloc[:, 0].values
X_val = data_test.iloc[:, 1].values
y_val = y_val + 1

In [23]:
new_reviews_test = []
for idx,data in enumerate(X_val):
    text = rdrsegmenter.tokenize(data)
    text = ' '.join([' '.join(x) for x in text])
    new_reviews_test.append(text)
X_val = new_reviews_test

In [24]:
print(len(y_val))
len(X_val)

# from sklearn.model_selection import train_test_split



1050


1050

In [25]:
X = [*X_train, *X_val]
y = [*y_train, *y_val]


In [26]:
# train_sents = X_train
# train_labels = y_train
# val_sents = X_val
# val_labels = y_val


**Train test split**

In [29]:
from sklearn.model_selection import train_test_split

train_sents, val_sents, train_labels, val_labels = train_test_split(X, y, test_size=0.3)

In [33]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.utils.generic_utils import get_custom_objects
from keras.layers import Activation
MAX_LEN = 195

train_ids = []
for sent in train_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    train_ids.append(encoded_sent)

val_ids = []
for sent in val_sents:
    subwords = '<s> ' + bpe.encode(sent) + ' </s>'
    encoded_sent = vocab.encode_line(subwords, append_eos=True, add_if_not_exist=False).long().tolist()
    val_ids.append(encoded_sent)
    
train_ids = pad_sequences(train_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
val_ids = pad_sequences(val_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")

In [34]:
train_masks = []
for idx,sent in enumerate(train_ids):
    mask = [int(token_id > 0) for token_id in sent]
    train_masks.append(mask)

val_masks = []
for sent in val_ids:
    mask = [int(token_id > 0) for token_id in sent]

    val_masks.append(mask)

**DataLoader**

In [35]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
train_inputs = torch.tensor(train_ids)
val_inputs = torch.tensor(val_ids)
train_labels = torch.tensor(train_labels)
val_labels = torch.tensor(val_labels)
train_masks = torch.tensor(train_masks)
val_masks = torch.tensor(val_masks)

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = SequentialSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=32)

# Train

In [36]:
from transformers import RobertaForSequenceClassification, RobertaConfig, AdamW

config = RobertaConfig.from_pretrained(
#     "./PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 3, output_hidden_states=False,
    "./PhoBERT_base_transformers/config.json", from_tf=False, num_labels = 3, output_hidden_states=False,
)
phobert = RobertaForSequenceClassification.from_pretrained(
    "./PhoBERT_base_transformers/model.bin",
    config=config
)

You are using a model of type bert to instantiate a model of type roberta. This is not supported for all configurations of models and can yield errors.
Some weights of the model checkpoint at ./PhoBERT_base_transformers/model.bin were not used when initializing RobertaForSequenceClassification: ['lm_head.decoder.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'roberta.pooler.dense.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenc

In [37]:
from torch import nn

class RobertaForAIViVN(torch.nn.Module):
    def __init__(self, num_labels):
        super(RobertaForAIViVN, self).__init__()
        self.num_labels = num_labels
        self.config = config
        self.bert_classifier = phobert
        
#         self.qa_outputs = nn.Linear(4*self.config.hidden_size, self.num_labels)
        self.qa_outputs = nn.Linear(self.config.hidden_size, self.num_labels)

    def forward(self, input_ids=None, token_type_ids=None, attention_mask=None, labels=None):
        outputs = self.bert_classifier(input_ids=input_ids,
                                    token_type_ids=None,
                                    attention_mask=attention_mask,
                                    labels=labels)
                                    
        cls_output = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
        logits = self.qa_outputs(cls_output)
        return logits

In [38]:
# BERT_SA = RobertaForAIViVN(num_labels=3)
BERT_SA = phobert
BERT_SA.cuda()
print('Done')

Done


In [39]:
import numpy as np
from sklearn.metrics import f1_score, accuracy_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    F1_score = f1_score(pred_flat, labels_flat, average='macro')
    
    return accuracy_score(pred_flat, labels_flat), F1_score

In [40]:
import random
import os 
import torch.nn.functional as F

os.environ["CUDA_VISIBLE_DEVICES"]="1"

from tqdm import tqdm_notebook
device = 'cuda'
epochs = 50
max = 0

param_optimizer = list(BERT_SA.named_parameters())
no_decay = ['bb_input_idsias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=5e-6, correct_bias=False)
# optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, correct_bias=False)


for epoch_i in range(0, epochs):
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    total_loss = 0
    BERT_SA.train()
    train_accuracy = 0
    nb_train_steps = 0
    train_f1 = 0
    
    for step, batch in tqdm_notebook(enumerate(train_dataloader)):
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        BERT_SA.zero_grad()
        outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask, 
            labels=b_labels)
        loss = outputs[0]
        total_loss += loss.item()
        
        logits = outputs[1].detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_train_accuracy, tmp_train_f1 = flat_accuracy(logits, label_ids)
        train_accuracy += tmp_train_accuracy
        train_f1 += tmp_train_f1
        nb_train_steps += 1
        ##############
#         loss =  F.binary_cross_entropy_with_logits(outputs[1],b_labels)
#         loss = loss.mean()
        loss.backward()
        #########
#         loss.backward()
        torch.nn.utils.clip_grad_norm_(BERT_SA.parameters(), 1.0)
        optimizer.step()
        
    avg_train_loss = total_loss / len(train_dataloader)
    print(" Accuracy: {0:.4f}".format(train_accuracy/nb_train_steps))
    print(" F1 score: {0:.4f}".format(train_f1/nb_train_steps))
    print(" Average training loss: {0:.4f}".format(avg_train_loss))
    BERT_SA.save_pretrained('./modelbert/{}'.format(epoch_i+1))
    print("Running Validation...")
    BERT_SA.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    eval_f1 = 0
    list_f1_score = []
    for batch in tqdm_notebook(val_dataloader):

        batch = tuple(t.to(device) for t in batch)

        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            outputs = BERT_SA(b_input_ids, 
            token_type_ids=None, 
            attention_mask=b_input_mask)
            logits = outputs[0]
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            tmp_eval_accuracy, tmp_eval_f1 = flat_accuracy(logits, label_ids)
            list_f1_score.append(tmp_eval_f1)
            eval_accuracy += tmp_eval_accuracy
            eval_f1 += tmp_eval_f1
            nb_eval_steps += 1
    if len(list_f1_score):
        print("F1 score:", sum(list_f1_score)/len(list_f1_score))
    if max < eval_accuracy/nb_eval_steps:
      max = eval_accuracy/nb_eval_steps
      e = epoch_i
    print('logits:    {}'.format(np.argmax(logits, axis=1).flatten()))
    print('labels_id: {}'.format(label_ids))
    print(" Accuracy: {0:.4f}".format(eval_accuracy/nb_eval_steps))
    print(" F1 score: {0:.4f}".format(eval_f1/nb_eval_steps))
print("Training complete!")
print('Best Valid acc : {}, epoch: {}'.format(max, e+1))

======== Epoch 1 / 50 ========
Training...


/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/ipykernel_launcher.py:33: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


0it [00:00, ?it/s]

 Accuracy: 0.4992
 F1 score: 0.4624
 Average training loss: 0.9884
Running Validation...


/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/ipykernel_launcher.py:72: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

0it [00:00, ?it/s]

 Accuracy: 0.9956
 F1 score: 0.9952
 Average training loss: 0.0143
Running Validation...


/root/miniconda3/envs/sentiment/lib/python3.7/site-packages/ipykernel_launcher.py:72: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/70 [00:00<?, ?it/s]

F1 score: 0.7915088538416913
logits:    [1 1 2 0 0 0 0 0 1 0 0 1 2 0 0 1 0 1 1 1 1 1 0 2 0 1 2 1 1 2]
labels_id: [1 1 0 0 0 0 0 0 1 1 0 1 2 0 0 2 0 1 1 1 0 0 0 2 0 0 2 1 0 2]
 Accuracy: 0.7971
 F1 score: 0.7915
Training complete!
Best Valid acc : 0.7971130952380953, epoch: 50


# Save model